**1 Подгрузим данные и необходимые библиотеки**

In [1]:
# библиотека для скачивания данных с гугл.диска
!pip install --upgrade --no-cache-dir gdown

In [2]:
# библиотека для векторизации текста, обновим до актуальной версии
!pip install -U gensim

     |████████████████████████████████| 24.1 MB 82.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import gdown
import pandas as pd
import numpy as np
import random
import string
from tqdm import tqdm_notebook as tqdm
import re
from scipy import spatial
import heapq
import timeit
from ast import literal_eval

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
pd.set_option('display.max_columns', None)

Файлы были взяты из этой задачки: kaggle.com/datasets/rounakbanik/the-movies-dataset

In [5]:
!gdown 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
!gdown 1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC

Downloading...
From: https://drive.google.com/uc?id=1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:01<00:00, 23.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
To: /content/ratings.csv
100% 710M/710M [00:08<00:00, 86.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC
To: /content/credits.csv
100% 190M/190M [00:02<00:00, 79.9MB/s]


**2 Проведем первичную обработку данных**

В первом датасете, которым мы воспользуемся, содержится вся основная информация о фильмах, например: бюджет, жанровая принадлежность, язык оригинала, название, описание, дата создания и тд.

In [6]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [8]:
# удалим ошибочные значения
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '1997-08-20'].index, inplace=True)

In [9]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2012-09-29'].index, inplace=True)

In [10]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2014-01-01'].index, inplace=True)

В этом наборе мы сосредоточимся на текстовых данных и  будем работать только со стобцами original_title (название фильма) и overview (официальное описание фильма)

In [11]:
# оставим только колонки, которые мы собираемся использовать
movies_metadata = movies_metadata[['id', 'title', 'overview', 'genres', 'production_companies']]

In [12]:
# заполняем пустые значения
movies_metadata = movies_metadata.fillna('')

In [13]:
# удалим дубли, потому что они есть и очень мешаются
movies_metadata = movies_metadata.drop_duplicates().reset_index(drop=True)

In [14]:
movies_metadata = movies_metadata.rename(columns={'id':'movieId'})

In [15]:
# меняем тип данных колонки, чтобы проще было джойниться с другой таблицей
movies_metadata['movieId'] = movies_metadata['movieId'].astype(int)

Во втором наборе содержится информация о съемочной команде, нас будет интересовать имя режисера

In [16]:
credits = pd.read_csv('credits.csv')[['crew', 'id']]

In [17]:
credits = credits.rename(columns={'id':'movieId'})

In [18]:
credits = credits.drop_duplicates().reset_index(drop=True)

In [19]:
credits.head(5)

,crew,movieId
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Теперь поработаем с еще одним набором - пользовательскими оценками фильмов

In [20]:
ratings = pd.read_csv('ratings.csv')

# выберем только достаточно высоко оцененные фильмы
ratings = ratings[ratings['rating'] > 3]

После подгрузки всех необходимых данных найдем их пересечение по айди фильмов

Дело в том, что не для всех фильмов есть данные о просмотрах и оценках, а для некоторых оценок нет данных о самом фильме

In [21]:
common_ids = set(movies_metadata.movieId).intersection(set(ratings.movieId))

ratings = ratings[ratings.movieId.isin(common_ids)].reset_index(drop=True)
movies_metadata = movies_metadata[movies_metadata.movieId.isin(common_ids)].reset_index(drop=True)

In [22]:
movies_metadata.shape[0]

6835

In [23]:
movies_metadata.groupby('movieId').count().sort_values(by='genres', ascending=False).head(4)

,title,overview,genres,production_companies
movieId,,,,
2,1,1,1,1
65300,1,1,1,1
65216,1,1,1,1
65188,1,1,1,1


In [24]:
credits = credits[credits.movieId.isin(common_ids)].reset_index(drop=True)

In [25]:
# здесь мы видим, что размерность не совпадает - есть подозрительные записи
credits.shape[0]

6837

In [26]:
# ищем задвоенные, но не идентичные записи
credits.groupby('movieId').count().sort_values(by='crew', ascending=False).head(4)

,crew
movieId,
159849,2
4912,2
64650,1
65216,1


In [27]:
credits[credits['movieId'] == 159849]

,crew,movieId
3181,"[{'credit_id': '52fe4c229251416c910f10d5', 'de...",159849
6056,"[{'credit_id': '52fe4c229251416c910f10d5', 'de...",159849


In [28]:
credits.drop(credits[credits.index == 3181].index, inplace=True)

In [29]:
credits[credits['movieId'] == 4912]

,crew,movieId
1525,"[{'credit_id': '52fe43e2c3a36847f80760b5', 'de...",4912
5856,"[{'credit_id': '52fe43e2c3a36847f80760a9', 'de...",4912


In [30]:
credits.drop(credits[credits.index == 5856].index, inplace=True)

In [31]:
# убеждаемся, что получили нужный размер
credits.shape[0]

6835

In [32]:
# соединяем с основным набором
movies_metadata = pd.merge(movies_metadata, credits, on='movieId')

In [33]:
movies_metadata.head(4) 

,movieId,title,overview,genres,production_companies,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,949,Heat,"Obsessive master thief, Neil McCauley leads a ...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'credit_id': '52fe4292c3a36847f802916d', 'de..."
3,710,GoldenEye,James Bond must unmask the mysterious head of ...,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","[{'name': 'United Artists', 'id': 60}, {'name'...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de..."


Cтолбцы genres, production_companies и crew имеют сложную структуру, но по факту их тип - строка, надо это исправить, чтобы достать из них необходимую нам информацию

In [34]:
type(movies_metadata['genres'][0])

str

In [35]:
# literal_eval позволяет восстановить исходную структуру признака из строки
movies_metadata['crew'] = movies_metadata['crew'].apply(literal_eval)
movies_metadata['genres'] = movies_metadata['genres'].apply(literal_eval)
movies_metadata['production_companies'] = movies_metadata['production_companies'].apply(literal_eval)

In [36]:
# получилось
type(movies_metadata['genres'][0])

list

Напишем и применим функции, достающие имена режисеров, названия студий и жанры фильмов

In [37]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [38]:
def get_genres_company(x):
    for i in x:
        return i['name']
    return np.nan

In [39]:
movies_metadata['director'] = movies_metadata['crew'].apply(get_director)
movies_metadata['genres'] = movies_metadata['genres'].apply(get_genres_company)
movies_metadata['production_company'] = movies_metadata['production_companies'].apply(get_genres_company)

In [40]:
movies_metadata = movies_metadata[['movieId', 'title', 'overview', 'director', 'genres', 'production_company']]

In [41]:
movies_metadata.head(4)

,movieId,title,overview,director,genres,production_company
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",John Lasseter,Animation,Pixar Animation Studios
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,Joe Johnston,Adventure,TriStar Pictures
2,949,Heat,"Obsessive master thief, Neil McCauley leads a ...",Michael Mann,Action,Regency Enterprises
3,710,GoldenEye,James Bond must unmask the mysterious head of ...,Martin Campbell,Adventure,United Artists


**3 С помощью обработки текста получим матрицу эмбедингов для каждого фильма**

Нам нужно перевести текстовые данные из названия и описания фильмов в формат, которым может оперировать компьютер - вектора эмбедингов. Перед векторизацией нужно предобработать данные и токенизировать их

После этого мы сможем выполнять операции над этими данными, например сравнивать между собой и находить расстояние между двумя фильмами

In [42]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    '''
       на вход: 
       - data - датафрейм с описанием и названием фильмов
       - lemmatizer - класс для лемматизации текстов
       - stop_words - стоп-слова
      
       на выходе: 
       - список с предобработанными текстовыми данными
    '''
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]

        # приводим к нижнему регистру и дропаем стоп-слова
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    
    return new_text

In [43]:
nltk.download("stopwords")

text_overview = preprocessing(movies_metadata["overview"])
text_title = preprocessing(movies_metadata["title"])
text_dir = preprocessing(movies_metadata["director"])
text_genres = preprocessing(movies_metadata["genres"])
text_company = preprocessing(movies_metadata["production_company"])

movies_metadata["overview"] = text_overview
movies_metadata["title"] = text_title 
movies_metadata["director"] = text_dir
movies_metadata["genres"] = text_genres
movies_metadata["production_company"] = text_company

# закидываем предобработанные описания и названия фильмов в общий список
text_prepr = text_overview + text_title + text_dir + text_genres + text_company

# и разбиваем на отдельные слова
text_prepr = [i.split() for i in text_prepr]
text_overview = [i.split() for i in text_overview]
text_title = [i.split() for i in text_title]
text_dir = [i.split() for i in text_dir]
text_genres = [i.split() for i in text_genres]
text_company = [i.split() for i in text_company]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

In [44]:
# с помощью word2vec получаем векторизованный словарь
model = Word2Vec(text_prepr, 
                 min_count=2)

model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 16575


После обработки всех текстовых данных перейдем к созданию матрицы эмбедингов для каждого фильма

In [45]:
def embedding_matrix(text, model):
    
    """
       На вход: словарь tokenizer.word_index 
       модель word2vec
        
       На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((id_film, 100))

    for sentence in text:
        for word in sentence:
            if word in model:
                embedding_matrix[text.index(sentence)] = model[word]

    return embedding_matrix

In [46]:
id_film = movies_metadata['movieId'].nunique()

# получим матрицы с эмбедингами для всех текстовых признаков
matrix_emb_over = embedding_matrix(text_overview, model.wv)
matrix_emb_title = embedding_matrix(text_title, model.wv)
matrix_emb_dir = embedding_matrix(text_dir, model.wv)
matrix_emb_genres = embedding_matrix(text_genres, model.wv)
matrix_emb_company = embedding_matrix(text_company, model.wv)

# и соединим их
matrix_emb = pd.concat([movies_metadata[['movieId']],
                        pd.DataFrame(matrix_emb_over),
                        pd.DataFrame(matrix_emb_title),
                        pd.DataFrame(matrix_emb_dir),
                        pd.DataFrame(matrix_emb_genres),
                        pd.DataFrame(matrix_emb_company)], axis=1)

In [47]:
matrix_emb.shape

(6835, 501)

In [48]:
matrix_emb.head(4)

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,862,0.556068,1.615709,0.876059,0.007163,-0.405107,-1.409601,-0.637801,1.078852,-0.265402,0.316124,-0.525692,-0.519314,-0.098338,-0.117680,0.141245,-0.178555,1.536127,-0.744872,0.024324,-1.547445,0.602023,1.286899,0.308311,-1.033199,-0.623717,0.394776,-1.706885,-0.507169,0.188869,0.425520,0.289110,0.583578,0.023909,-1.097242,-0.337803,1.868933,-0.117101,-0.909175,-0.461754,-2.202978,0.205635,-0.858441,0.324831,0.719995,-0.497803,-0.945199,-0.877866,-0.783940,-0.256061,0.222129,-0.095393,-0.825684,-0.457758,-0.776483,-0.112816,-0.322046,-0.358678,-0.275679,-1.224805,0.524271,-0.003656,0.097169,-0.152167,1.113386,0.202797,0.730545,0.561966,-0.648223,-1.136990,0.216341,-2.230139,-0.664834,0.530666,-0.589671,1.044388,0.038183,-0.228987,-0.123846,-0.007544,0.290982,-0.507860,0.986863,-0.500667,1.651892,-0.603573,0.160012,0.594636,1.641553,0.818940,-0.062325,0.731453,-0.897390,0.576642,-0.368526,0.705777,-0.629680,-0.840384,0.400273,-0.312692,0.733484,-0.018345,0.812138,0.602261,0.098497,-0.126435,-1.349455,0.086711,1.067185,-0.194448,-0.387576,-0.197748,-1.152488,-0.088929,0.000620,0.523813,-0.360912,0.392191,-1.275492,0.454085,-0.941449,0.682680,0.631491,0.55187,-0.270034,-0.306306,0.491714,-1.039287,-0.206685,-0.463767,0.170055,0.239922,0.295040,-0.063665,-0.472681,-0.227273,0.974464,-0.185223,-0.767872,-0.602776,-1.248213,0.002821,-0.496229,-0.061598,0.068942,0.206970,-0.452563,-0.668146,-0.083131,0.168204,0.511074,0.217822,-0.467949,-0.203695,-0.288213,-0.433458,0.427084,0.301388,0.227683,-0.335996,0.010810,-0.051009,0.297812,-0.312817,-0.016873,-0.440907,0.410169,-0.144951,0.080203,-0.825063,0.662923,-0.852319,0.327230,0.826202,-0.487361,0.739000,0.238545,-0.022461,-0.137056,-0.672108,0.226768,-0.653074,0.134347,-0.399047,1.182680,-0.061523,-0.046876,-0.097715,0.976032,0.838907,0.276402,0.840872,-0.211500,0.177922,-0.000264,0.724299,0.075955,0.187001,-0.409974,0.144396,0.257436,0.005831,0.014206,0.022763,0.007051,-0.003065,-0.047710,-0.006497,0.033551,0.001887,-0.015767,-0.013099,-0.036468,-0.011593,-0.011040,0.024422,-0.009509,0.006805,-0.034958,0.002348,-0.038966,0.013644,0.014958,0.020382,-0.013590,-0.009595,0.016570,-0.023071,-0.013608,-0.013527,0.003606,0.000506,0.013166,0.011365,-0.001896,-0.015809,0.032585,-0.015266,-0.021320,-0.014446,-0.031021,0.006594,-0.003658,-0.007574,-0.002658,0.012429,-0.005065,-0.014041,0.005986,0.016757,0.010259,0.003415,-0.016547,-0.004268,-0.013415,-0.008321,0.016269,0.001241,0.003285,-0.003083,0.003052,0.003247,0.012347,-0.010893,-0.007552,-0.004302,0.013879,

**4 Напишем аналог функции train_test_split для пользователя и его предпочтений**


Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях

In [49]:
def train_test_split_films(dataframe, user_list, size=0.7):

  '''
     на вход подается:
     - dataframe - название датафрейма с рейтингами
     - user_list - список пользователей
     - size - пропорция разделения

     на выходе для всех пользователей получаем один словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tts_data_dict = {}

  # итерируемся по всем пользователям
  for user in user_list:

    # выбираю данные по нужному юзеру и перемешиваю
    temp_df = dataframe[dataframe['userId'] == user] \
              .sample(dataframe[dataframe['userId'] == user].shape[0]) \
              [['movieId', 'rating']]

    # отфильтровываем пользователей с количеством просмотров < 10
    if temp_df.shape[0] < 10:
      continue

    # вычисляю длину тренировочного набора для нужного юзера
    shape = int(temp_df.shape[0]*size)

    # сразу записываем в словарь четыре вектора
    user_tts_data_dict[user] = [temp_df.iloc[:shape]['movieId'].tolist(),
                                temp_df.iloc[:shape]['rating'].tolist(),
                                temp_df.iloc[shape:]['movieId'].tolist(),
                                temp_df.iloc[shape:]['rating'].tolist()]

  return user_tts_data_dict

In [50]:
# пользователей слишком много, мы (в силу отсутствия мощностей) не сможем 
# построить предсказания по всем, но можно выбрать сэмпл
len(ratings['userId'].unique())

258025

In [51]:
# вторым параметром на вход можно подать любой список id пользователей
splitted_data = train_test_split_films(ratings, ratings['userId'].unique()[:1000], 0.7)

Мы получили словарь, в котором для каждого пользователя, который посмотрел как минимум 10 фильмов, есть тренировочный набор фильмов и оценки к ним + тестовый набор фильмов и оценки к ним

**5 Для каждого пользователя по train-фильмам находим средний вектор предпочтений**

In [52]:
def get_avg_weightened_vec(dict_name, movies_emb_df):

  '''
     на вход подается:
     - dict_name - название словаря с данными после сплита 
     - movies_emb_df - название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор для каждого зрителя

     на выходе:
     - словарь с ключами - айди пользователей и значениями - средним взвешенным 
     вектором по просмотренным фильмам в train-наборе по каждому пользователю
  '''

  user_vec_data_dict = {}

  # итерируемся по всем пользователям
  for key in dict_name.keys():

    # находим все вектора фильмов из train-выборки в эмбединг-матрице
    # и джойним с их рейтингами от выбранного пользователя
    temp_df = movies_emb_df \
              .loc[movies_emb_df['movieId'].isin(dict_name[key][0])] \
              .merge(pd.DataFrame(dict_name[key][0], dict_name[key][1], 
                                  columns=['movieId'])
                                  .reset_index()
                                  .rename(columns={'index':'rating'}), on='movieId')

    # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
    avg_weightened_vec = list(temp_df \
                              .drop(['movieId'], axis=1)
                              .mul(temp_df['rating'], axis=0)
                              .drop(['rating'], axis=1)
                              .sum()
                              .mul(1 / temp_df['rating'].sum(), axis=0))
    
    # все вектора записываем в словарь под ключ-айди пользователя
    user_vec_data_dict[key] = avg_weightened_vec

  return user_vec_data_dict

In [53]:
# получим матрицу пользователей
res_vec_dict = get_avg_weightened_vec(splitted_data, matrix_emb)

In [54]:
# преобразуем к табличному виду
res_vec_df = pd.DataFrame.from_dict(res_vec_dict, orient='index').reset_index().rename(columns={'index':'Id'})

In [55]:
res_vec_df.head(4)

,Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,1,-0.011159,0.238254,0.178978,0.031502,-0.031520,-0.391880,0.021327,0.321259,-0.060795,-0.124310,-0.045887,-0.343996,-0.025785,-0.002130,0.154371,-0.108889,0.104831,-0.390073,0.126068,-0.278083,0.205441,0.175021,0.161070,-0.079607,-0.084809,0.141978,-0.291209,-0.058128,-0.135413,0.047643,0.060645,0.076699,-0.010158,-0.132849,-0.075410,0.295337,-0.045710,-0.233590,-0.179597,-0.359401,-0.004614,-0.135479,-0.022020,0.020756,0.069039,-0.130958,-0.193630,-0.024366,0.050195,0.153840,0.071458,-0.135411,-0.059884,-0.093315,-0.129589,0.117773,0.086857,0.072536,-0.097528,0.004870,-0.013058,0.084598,-0.097460,-0.012596,-0.130980,0.122814,-0.040461,0.031242,-0.240570,0.199046,-0.248634,0.105794,0.242987,-0.132256,0.209148,0.064158,-0.001685,-0.045101,-0.195636,0.069406,-0.191880,0.037242,-0.120704,0.346481,-0.015971,-0.018170,-0.026539,0.267133,0.249956,0.086020,0.254894,-0.052406,0.050308,0.002198,0.224807,0.023187,0.058302,-0.129706,0.046041,0.072895,-0.008731,0.198921,0.138847,0.025926,-0.024602,-0.318891,0.019175,0.263673,-0.050498,-0.101364,-0.045192,-0.277875,-0.014804,0.000998,0.118465,-0.095843,0.084759,-0.312918,0.099672,-0.223375,0.159494,0.145447,0.138801,-0.059691,-0.071316,0.114955,-0.246558,-0.048011,-0.111243,0.038384,0.050047,0.061591,-0.013763,-0.107957,-0.058991,0.238141,-0.035484,-0.185005,-0.141670,-0.293840,-0.005481,-0.115482,-0.021605,0.011570,0.054501,-0.100783,-0.157123,-0.021699,0.038388,0.120646,0.058453,-0.115845,-0.052817,-0.076605,-0.110876,0.098109,0.075902,0.059810,-0.086141,0.007769,-0.012829,0.075168,-0.077228,-0.005392,-0.102004,0.100468,-0.032126,0.023694,-0.191134,0.164468,-0.200799,0.081955,0.199882,-0.112453,0.176468,0.053151,-0.004323,-0.035009,-0.157709,0.057734,-0.151185,0.032695,-0.093247,0.280113,-0.011214,-0.019348,-0.019970,0.229592,0.195422,0.068615,0.208426,-0.038615,0.046991,0.002321,0.185087,0.017639,0.049420,-0.103754,0.032895,0.064977,0.002954,0.088857,0.062879,0.009322,-0.016122,-0.137749,0.006033,0.111887,-0.023758,-0.042019,-0.018302,-0.120381,-0.010110,-0.002219,0.055363,-0.039568,0.042045,-0.128901,0.046490,-0.099501,0.0743

Теперь у нас для каждого пользователя готов вектор его предпочтений, который можно сравнить с фильмами и порекомендовать наиболее подходящие

**6 Найдем топ близких фильмов, которые пользователь еще не смотрел**

Мы будем использовать косинусное расстояние между вектором предпочтений пользователя и векторами фильмов. Косинусное расстояние – это мера похожести двух векторов. Скалярное произведение векторов и косинус угла θ между ними связаны следующим соотношением: a * b = ||a|| ||b|| cosθ. Имея два вектора A и B, получаем косинусное расстояние – cos(θ)

In [56]:
def get_closest_films(usr_avg_vec_df, user_id, films_emb_mtrx, user_data_dict, top_n=15):

  '''
     на вход подается:
     - usr_avg_vec_df - матрица с train-предпочтениями пользователя
     - user_id - айди пользователя, для которого мы хотим получить рекомендации
     - films_emb_mtrx - эмбединг матрица фильмов
     - splitted_data_dict - словарь с train_test data для всех пользователей
     - top_n - количество рекомендуемых фильмов

     на выходе:
     - список наиболее подходящих непросмотренных фильмов
  '''

  scores = {}

  # достаем средний вектор предпочтений пользователя
  user_avg_vec = usr_avg_vec_df[usr_avg_vec_df['Id'] == user_id] \
                                .drop('Id', axis=1) \
                                .values \
                                .flatten() \
                                .tolist()

  # итерируемся по списку фильмов и находим косинусное расстояние между
  # средним вектором пользователя и каждым фильмом
  for film in films_emb_mtrx['movieId']:

    film_vec = films_emb_mtrx[films_emb_mtrx['movieId'] == film] \
                              .drop('movieId', axis=1) \
                              .values \
                              .flatten() \
                              .tolist()

    # промежуточные результаты записываем в словарь
    scores[film] = 1 - spatial.distance.cosine(user_avg_vec, film_vec)

  # удаляем из словаря фильмы, которые пользователь уже смотрел (в train-наборе)
  for viewed_film in user_data_dict[user_id][0]:
    del scores[viewed_film]

  # возвращаем топ фильмов в порядке убывания близости к вектору пользователя
  # записываем их в пятый список под айди пользователя в словаре с train-test-data
  user_data_dict[user_id].append(heapq.nlargest(top_n, scores, key=scores.get))

In [57]:
# дополним словарь предсказаниями по каждому пользователю
for i, user in enumerate(splitted_data.keys()):
  print('Running:', i + 1, '/', len(splitted_data))
  get_closest_films(res_vec_df, user, matrix_emb, splitted_data)

Running: 1 / 544


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Running: 2 / 544
Running: 3 / 544
Running: 4 / 544
Running: 5 / 544
Running: 6 / 544
Running: 7 / 544
Running: 8 / 544
Running: 9 / 544
Running: 10 / 544
Running: 11 / 544
Running: 12 / 544
Running: 13 / 544
Running: 14 / 544
Running: 15 / 544
Running: 16 / 544
Running: 17 / 544
Running: 18 / 544
Running: 19 / 544
Running: 20 / 544
Running: 21 / 544
Running: 22 / 544
Running: 23 / 544
Running: 24 / 544
Running: 25 / 544
Running: 26 / 544
Running: 27 / 544
Running: 28 / 544
Running: 29 / 544
Running: 30 / 544
Running: 31 / 544
Running: 32 / 544
Running: 33 / 544
Running: 34 / 544
Running: 35 / 544
Running: 36 / 544
Running: 37 / 544
Running: 38 / 544
Running: 39 / 544
Running: 40 / 544
Running: 41 / 544
Running: 42 / 544
Running: 43 / 544
Running: 44 / 544
Running: 45 / 544
Running: 46 / 544
Running: 47 / 544
Running: 48 / 544
Running: 49 / 544
Running: 50 / 544
Running: 51 / 544
Running: 52 / 544
Running: 53 / 544
Running: 54 / 544
Running: 55 / 544
Running: 56 / 544
Running: 57 / 544


Мы получили топ n предсказаний для каждого пользователя, теперь можно переходить к оценки результатов по основным метрикам

**7 Напишем функции для метрик и оцениваем алгоритм**

Precision можно интепретировать как долю релевантных рекомендаций, мы дополнительно считаем среднее по всем пользователям

Precision = TP / (TP + FP), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FP - это все рекомендованные объекты, то есть K - количество предсказанных объектов

In [58]:
def mean_precision(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика precision@k по всем пользователям
  '''

  sum_precision = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    k = len(user_data_dict[user][4])

  # находим среднее по пользователям
  return sum_precision / len(user_data_dict)

Recall интерпретируется как доля релевантных объектов, попавших в рекомендации, мы дополнительно считаем среднее по всем пользователям

Recall = TP / (TP + FN), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FN - это все объекты из тестовой выборки

In [59]:
def mean_recall(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика recall@k по всем пользователям
  '''

  sum_recall = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    TP_FN = len(user_data_dict[user][2])
    
    sum_recall += TP / TP_FN

  # находим среднее по пользователям
  return sum_recall / len(user_data_dict)

Mean Reciprocal Rank - средний обратный ранк первого правильного вхождения рекомендации в тестовые данные

In [60]:
def MRR(user_data_dict):

  '''
     RR - обратный ранг, 1 / индеск первого вхождения рекомендации в тест

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MRR@k по всем пользователям
  '''

  sum_rr = 0

  # находим порядковый номер первого вхождения правильной рекомендации в тест
  for user in user_data_dict.keys():
    rank = [1 + splitted_data[user][4].index(x) for x in \
            set(splitted_data[user][4]).intersection(set(splitted_data[user][2]))]
    
    # находим обратный ранк, если попадания нет, то зануляем
    if len(rank) == 0:
      rr = 0
    else:
      rr = 1 / min(rank)

    sum_rr += rr

  # находим среднее по пользователям
  return sum_rr / len(user_data_dict)

Average Precision дополнительно вознаграждает нас за предварительну загрузку рекомендаций, которые будут правильными, то есть здесь, в отличие от Precision, учитывается порядок выдаваемых рекомендаций и их точность отдельно. MAP - это среднее AP по всем пользователям

In [61]:
def mean_average_precision_at_k(user_data_dict):

  '''
     Mean Average Precision - средняя точность по пользователям

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MAP@k по всем пользователям
  '''

  sum_ap = 0

  # итерируемся по пользователям
  for user in user_data_dict.keys():

    num_hits = 0
    score = 0

    for i, p in enumerate(user_data_dict[user][4]):

      if p in user_data_dict[user][2] and p not in user_data_dict[user][4][:i]:
        num_hits += 1
        score += num_hits / (i + 1)

    sum_ap += score / min(len(user_data_dict[user][2]), len(user_data_dict[user][4]))

  # находим среднее по пользователям
  return sum_ap / len(user_data_dict)

**Результат:**

In [62]:
print('Mean Precision: ', round(mean_precision(splitted_data), 6))
print('Mean Recall: ', round(mean_recall(splitted_data), 6))
print('Mean Reciprocal Rank:', round(MRR(splitted_data), 6))
print('Mean Average Precision at K:', round(mean_average_precision_at_k(splitted_data), 6))

Mean Precision:  0.0
Mean Recall:  0.003653
Mean Reciprocal Rank: 0.010193
Mean Average Precision at K: 0.001157
